# Part 2: Asymmetric encryption using Rivest-Shamir-Adleman (RSA)

## Task 1: <font color='gray'>How _NOT_ to do it</font>

Try **to figure out the issues** in the following code that uses RSA API. The code itself might not even be runable.

Yo can look at the following links if you want:

- [cryptography.rsa](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=rsa#) module documentation
- [PKCS1v15 padding caveats](https://cryptography.io/en/latest/limitations/?highlight=PKCS1%20v1.5#rsa-pkcs1-v1-5-constant-time-decryption)
- [Cryptography right answers](https://www.daemonology.net/blog/2009-06-11-cryptographic-right-answers.html)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding

# Generate a private key of size 512
private_key = rsa.generate_private_key(
    public_exponent=3,
    key_size=512,
)

# Our message to encrypt (quite long)
message = b"message" * 100

# Obtain the public key from the private one
public_key = private_key.public_key()

# Use PKCS#1 v1.5 padding
pad = padding.PKCS1v15()

# Encrypt the message
ciphertext = public_key.encrypt(
    message,
    pad,
)

In [ ]:
# Issue #1: `65537` is recommended as the public exponent
# Issue #2: keys shorter than 2048 bits are not generally considered secure

private_key = rsa.generate_private_key(
    public_exponent=3,
    key_size=512,
)

message = b"message" * 100
public_key = private_key.public_key()

# Issue #3: PKCS1v15 is not the recommended padding, especially in the
# combination with public exponent `3`
pad = padding.PKCS1v15()

# Issue #4: encrypting messages longer than the modulus is not possible
ciphertext = public_key.encrypt(
    message,
    pad
)

# More on:
# combination of `PKCS1v15` and `public_exponent=3` is known to be
# vulnerable to some attacks (search for PKCS):
# https://www.daemonology.net/blog/2009-06-11-cryptographic-right-answers.html

## <font color='blue'>_Checkpoint 1_:</font> <font color='gray'>encryption issues</font>

**DO NOT** continue further in the notebook, wait for the tutor
to tell you :-). Don't spoil the seminar for yourself.

## Task 2: <font color='gray'>How to DO it</font>

### Issues #1 and #2: Key generation

To generate a secure key, we need to use bigger key sizes and follow [**the recommendations**](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=verify#cryptography.hazmat.primitives.asymmetric.rsa.generate_private_key)
for the public exponent.

In [ ]:
# Generate a 4096-bit long private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
)

public_key = private_key.public_key()

# The type of key is `rsa.RSAPrivateKey`, not `bytes`
print(isinstance(private_key, rsa.RSAPrivateKey))

# Similarly with the public key
print(isinstance(public_key, rsa.RSAPublicKey))

### Issues #3: Message (un)padding
Again, we need to follow [**the recommendations**](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=verify#cryptography.hazmat.primitives.asymmetric.padding.OAEP) and use a different padding scheme - OAEP.

In [ ]:
from cryptography.hazmat.primitives import hashes

# The padding scheme uses hash functions internally
pad = padding.OAEP(
    mgf=padding.MGF1(algorithm=hashes.SHA256()), 
    algorithm=hashes.SHA256(), 
    label=None
)

### Issue #4: Encrypting long messages
It is not mathematically possible to encrypt messages longer than the key modulus.

In [ ]:
# We can encrypt only short messages
message = b"message"

# Encrypt with the public key
ciphertext = public_key.encrypt(
    plaintext=message,
    padding=pad,
)

# To decrypt we use the private key
plaintext = private_key.decrypt(
    ciphertext=ciphertext,
    padding=pad,
)

print(plaintext == message)

## Task 3: <font color='gray'>Implement RSA encrypt/decrypt functions</font>

Consult the [`cryptography.rsa`](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/) module if necessary :)

In [ ]:
from typing import Optional


def rsa_encrypt(public_key: rsa.RSAPublicKey, message: bytes) -> bytes:
    # TODO: define the padding
    # TODO: try encrypting the message
    # TODO: return the ciphertext
    pass


def rsa_decrypt(private_key: rsa.RSAPrivateKey, ciphertext: bytes) -> Optional[bytes]:
    # TODO: define the same padding
    # TODO: try decrypting the message
    # TODO: return the ciphertext
    pass

In [ ]:
def rsa_encrypt(public_key: rsa.RSAPublicKey, message: bytes) -> Optional[bytes]:
    # TODO: define the padding
    pad = padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    )
    # TODO: encrypt the message
    try:
        ciphertext = public_key.encrypt(
            message,
            pad,
        )
    except:
        ciphertext = None
    # TODO: return the ciphertext
    return ciphertext


def rsa_decrypt(private_key: rsa.RSAPrivateKey, ciphertext: bytes) -> Optional[bytes]:
    pad = padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    )
    try:
        plaintext = private_key.decrypt(
            ciphertext,
            pad,
        )
    except:
        plaintext = None

    return plaintext

## <font color='blue'>_Checkpoint 2:_</font> <font color='gray'>webserver key API</font>

**DO NOT** continue futher in the notebook, wait for the tutor to tell you :-).<br>

Don't spoil the seminar for yourself.

## Task 4: <font color='gray'>Communication using RSA</font>

Contrary to symmetric ciphers we **do not need** a second channel
in order to share the secret key. However, we need to publish
our public key. Next to the previous `send_message`, `recv_message` functions
we have prepared also functions to publish your own key and to
fetch someone else's key:

```python
send_message(uco_from: int, uco_to: int, content: bytes) -> str

recv_message(uco: int) -> Mapping[str, Union[int, bytes]]

publish_key(uco: int, key: rsa.RSAPublicKey) -> str

fetch_key(uco: int) -> Optional[rsa.RSAPublicKey]
```

1. **Generate** your own private/public RSA keypair.
2. **Publish** your public key.
3. **Fetch** a friend's public key.
4. **Encrypt** a message for a friend and **send** it.
5. **Receive** a message from a friend and **decrypt** it.

In [ ]:
from server_communication import send_message, recv_message
from server_communication import publish_key, fetch_key

In [ ]:
help(publish_key)

In [ ]:
help(fetch_key)

In [ ]:
# TODO: 1. **Generate** your own private/public RSA keypair.
# Warning: do not overwrite your own key after you've published it
# by executing this cell again and again! :-)

In [ ]:
# TODO: 2. **Publish** your public key.
# TODO: 3. **Fetch** a friend's public key.
# TODO: 4. **Encrypt** a message for a friend and **send** it.
# TODO: 5. **Receive** a message from a friend and **decrypt** it.

In [ ]:
# TODO: 1. **Generate** your own private/public RSA keypair.
# Warning: do not overwrite your own key after you've published it
# by executing this cell again and again! :-)
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
)
public_key = private_key.public_key()

In [ ]:
# TODO: 2. **Publish** your public key.
publish_key(uco=408788, key=public_key)
# TODO: 3. **Fetch** a friend's public key.
# I've used my own uco, to make the example work without the second party
friends_key = fetch_key(uco=408788)
# TODO: 4. **Encrypt** a message for a friend and **send** it.
ciphertext = rsa_encrypt(public_key=friends_key, message=b"Hello darkness, my old friend.")
send_message(uco_from=408788, uco_to=408788, content=ciphertext)
# TODO: 5. **Receive** a message from a friend and **decrypt** it.
ciphertext = recv_message(uco=408788)[408788]
plaintext = rsa_decrypt(private_key=private_key, ciphertext=ciphertext)
print(plaintext)

## (Bonus) Task: <font color='gray'>Walkthrough "textbook" RSA</font>

When we use the RSA API from the `cryptography` module we
do not directly handle the primes $p,q$ and the private exponent
$d$. However, even if hidden, those values are still present.

### Refresh the math behind RSA encryption/decryption routines

RSA consists of **private** primes $p,q$ that form the modulus

\begin{align}\label{N}
N= p\cdot q
\end{align}

The **encryption** of a message $m$ is then defined as:

\begin{align*}
c=m^e \mod N
\end{align*}

where $e$ is the public exponent (usually $65537$) and $c$ is
the ciphertext. The **decryption** is done using the **private**
exponent $d$ that satisfies:

\begin{align}\label{priv_def}
e\cdot d \equiv 1 \mod \phi(N)
\end{align}

For $p,q$ primes it holds that

\begin{align*}
\phi(N) = \phi(p \cdot q) = \phi(p) \cdot \phi(q) = (p-1)(q-1)
\end{align*}

The **decryption** routine than looks like this

\begin{align}\label{decryption}
c^d = (m^e)^d \equiv m^{e\cdot d} \equiv m \mod N
\end{align}

Use the values before to verify the equalities/congruencies
(1), (2) and (3).

In [2]:
p = private_key.private_numbers().p
q = private_key.private_numbers().q
d = private_key.private_numbers().d

e = private_key.public_key().public_numbers().e
N = private_key.public_key().public_numbers().n

# we will generate a random message
import random

m = random.randint(0, N - 1)

# HINT: use the pow(base, exponent, modulus) to perform
# modular exponentiation quickly

# TODO: equation (1)

# TODO: equation (2)

# TODO: equation (3)

NameError: name 'private_key' is not defined